In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sqlalchemy import create_engine

pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

In [2]:
path = 'sqlite:///../data/processed/'

# Import protest and countries data (already merged)
engine = create_engine(path+'merged.db')
with engine.begin() as connection:
    df = pd.read_sql('SELECT * FROM merged', con=connection)

In [3]:
# Relace 'placeholder' NaN values, as defined by data dictionary (see raw data directory)
df.replace(-999, np.nan, inplace=True)
df.replace(-999.0, np.nan, inplace=True)
df.replace('-999', np.nan, inplace=True)
df.replace('-999.0', np.nan, inplace=True)
print(df.iloc[:, :100].info())
print(df.iloc[:, 100:].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15140 entries, 0 to 15139
Data columns (total 100 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  15140 non-null  int64  
 1   country                             15140 non-null  object 
 2   location                            15140 non-null  object 
 3   region                              15140 non-null  object 
 4   protestnumber                       15140 non-null  int64  
 5   protesterviolence                   15140 non-null  int64  
 6   protesteridentity                   15140 non-null  object 
 7   startyear                           15140 non-null  int64  
 8   startmonth                          15140 non-null  int64  
 9   startday                            15140 non-null  int64  
 10  endyear                             15140 non-null  int64  
 11  endmonth                            1514

In [4]:
df.head(3)

,id,country,location,region,protestnumber,protesterviolence,protesteridentity,startyear,startmonth,startday,endyear,endmonth,endday,start_datetime,end_datetime,duration_days,participants,participants_category_original,participants_category_manufactured,sources,notes,demand_labor-wage-dispute,demand_land-farm-issue,demand_police-brutality,demand_political-behavior/process,demand_price-increases/tax-policy,demand_removal-of-politician,demand_social-restrictions,response_accomodation,response_arrests,response_beatings,response_crowd-dispersal,response_ignore,response_killings,response_shootings,system,yrsoffc,finittrm,yrcurnt,termlimit,reelect,multpl,military,defmin,percent1,percentl,prtyin,execme,execrlc,execnat,execrurl,execreg,execrel,execage,allhouse,nonchief,totalseats,gov1me,gov1seat,gov1vote,gov1rlc,gov1nat,gov1rurl,gov1reg,gov1rel,gov1age,gov2me,gov2seat,gov2vote,gov2rlc,gov2nat,gov2rurl,gov2reg,gov2rel,gov2age,gov3me,gov3seat,gov3vote,gov3rlc,gov3nat,gov3rurl,gov3reg,gov3rel,gov3age,govoth,govothst,govothvt,opp1me,opp1seat,opp1vote,opp1rlc,opp1nat,opp1rurl,opp1reg,opp1rel,opp1age,opp2me,opp2seat,opp2vote,opp3me,opp3seat,opp3vote,oppoth,oppothst,oppothvt,ulprty,numul,ulvote,oppmajh,oppmajs,dateleg,dateexec,legelec,exelec,liec,eiec,mdmh,mdms,ssh,pluralty,pr,housesys,sensys,thresh,dhondt,cl,select,gq,gqi,fraud,auton,muni,state,author,stconst,gwno,numgov,numvote,numopp,oppvote,maj,partyage,herfgov,herfopp,herftot,frac,oppfrac,govfrac,tensys_strict,tensys,checks_lax,checks,stabs_strict,stabs,stabns_strict,stabns,tenlong_strict,tenlong,tenshort_strict,tenshort,polariz
0,201990001,Canada,national,North America,1,0,unknown,1990,1,15,1990,1,15,1990-01-15 00:00:00.000000,1990-01-15 00:00:00.000000,0,1000,unknown,1000-1999,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...,1,0,0,1,0,0,0,0,0,0,0,1,0,0,Parliamentary,6.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,6.0,PCP,Right,0.0,0.0,0.0,0.0,136.0,0.0,NaN,295.0,PCP,169.0,43.009998,Right,0.0,0.0,0.0,0.0,136.0,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,LPC,83.0,31.91,Left,0.0,0.0,0.0,0.0,NaN,NDP,43.0,20.379999,independent,1.0,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,7.0,7.0,1.0,888.0,0.28,1.0,0.0,Plurality,NaN,NaN,NaN,NaN,None,3.0,0.0,0.0,0.0,Legislature and executive locally elected,Legislature and executive locally elected,1.0,1.0,20.0,169.0,43.009998,127.0,52.459999,0.570945918560028,136.0,1.0,0.541819095611572,None,0.576225340366364,0.461817264556885,0.0,60.0,60.0,4.0,4.0,0.0,0.0,0.0,0.0,6.0,6.0,6.0,6.0,0.0
1,201990002,Canada,"Montreal, Quebec",North America,2,0,unknown,1990,6,25,1990,6,25,1990-06-25 00:00:00.000000,1990-06-25 00:00:00.000000,0,1000,unknown,1000-1999,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,Parliamentary,6.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,6.0,PCP,Right,0.0,0.0,0.0,0.0,136.0,0.0,NaN,295.0,PCP,169.0,43.009998,Right,0.0,0.0,0.0,0.0,136.0,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,LPC,83.0,31.91,Left,0.0,0.0,0.0,0.0,NaN,NDP,43.0,20.379999,independent,1.0,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,7.0,7.0,1.0,888.0,0.28,1.0,0.0,Plurality,NaN,NaN,NaN,NaN,None,3.0,0.0,0.0,0.0,Legislature and executive locally elected,Legislature and executive locally elected,1.0,1.0,20.0,169.0,43.009998,127.0,52.459999,0.570945918560028,136.0,1.0,0.541819095611572,None,0.576225340366364,0.461817264556885,0.0,60.0,60.0,4.0,4.0,0.0,0.0,0.0,0.0,6.0,6.0,6.0,6.0,0.0
2,201990003,Canada,"Montreal, Quebec",North America,3,0,separatist parti quebecois,1990,7,1,1990,7,1,1990-07-01 00:00:00.000000,1990-07-01 00:00:00.000000,0,500,unknown,100-999,1. quebec protest after queen calls for unity ...,"the queen, after calling on canadians to remai...",0,0,0,1,0,0,0,0,0,0,0,1,0,0,Parliamentary,6.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,6.0,PCP,Right,0.0,0.0,0.0,0.0,136.0,

In [8]:
# Set semi-arbitrary threshold for the maximum number of missing values to justify keeping
MAX_MISSING_VALUES = 100

# Copy main df to retain a full dataset
df_cut = df.copy()

# Determine the number of missing values in each column
na_counts_by_col = {}
for col in df_cut.columns:
    na_ct = df_cut[col].isna().sum()
    na_counts_by_col[col] = na_ct # Keep record in dictionary in case you want to investigate
    
    if na_ct > MAX_MISSING_VALUES:
        df_cut.drop(col, axis=1, inplace=True)
        
df_cut.dropna(inplace=True)
df_cut.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14853 entries, 0 to 15139
Data columns (total 51 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  14853 non-null  int64  
 1   country                             14853 non-null  object 
 2   location                            14853 non-null  object 
 3   region                              14853 non-null  object 
 4   protestnumber                       14853 non-null  int64  
 5   protesterviolence                   14853 non-null  int64  
 6   protesteridentity                   14853 non-null  object 
 7   startyear                           14853 non-null  int64  
 8   startmonth                          14853 non-null  int64  
 9   startday                            14853 non-null  int64  
 10  endyear                             14853 non-null  int64  
 11  endmonth                            14853

In [12]:
df_cut['numgov'] = df_cut['numgov'].astype('float').astype('int')
df_cut['numopp'] = df_cut['numopp'].astype('float').astype('int')
df_cut.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14853 entries, 0 to 15139
Data columns (total 51 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  14853 non-null  int64  
 1   country                             14853 non-null  object 
 2   location                            14853 non-null  object 
 3   region                              14853 non-null  object 
 4   protestnumber                       14853 non-null  int64  
 5   protesterviolence                   14853 non-null  int64  
 6   protesteridentity                   14853 non-null  object 
 7   startyear                           14853 non-null  int64  
 8   startmonth                          14853 non-null  int64  
 9   startday                            14853 non-null  int64  
 10  endyear                             14853 non-null  int64  
 11  endmonth                            14853

# Run it through a model!